# Data Project 1

In [25]:
# Import the required libraries
import requests
import pandas as pd
import sqlite3
import json

# Ask User for Data Source: CSV or API
while True:
    # Prompt the user to choose between CSV and API
    print("Choose the data source: 1 for CSV, 2 for API")

    # Get input from the user and store it in the "source_input" variable
    source_input = input("Enter your choice (1 or 2): ")

    # Check if the input is either "1" or "2"
    if source_input in ['1', '2']:
        # If valid, break out of the loop
        break
    else:
        # If invalid, inform the user and loop again
        print("Error: Invalid choice. Please enter 1 for CSV or 2 for API.")

# Initialize the DataFrame as None to hold the data once it's loaded from the source
df = None





# Load Data from CSV or API Based on User Choice

# If the user chooses CSV
if source_input == '1':

    # Define the path to the CSV file
    csv_file = 'Sleep_health_and_lifestyle_dataset.csv'

    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Check if the DataFrame is not empty
    if df is not None:

        # Display message with the number of records and columns
        print(f"CSV file loaded with {df.shape[0]} records and {df.shape[1]} columns.")
    else:
        # Error message where the DataFrame is empty
        print("Error: Issue with CSV file.")


# If the user chooses API
elif source_input == '2':

    # API URL for the OpenFDA food event API with search parameters and limit
    api_url = 'https://api.fda.gov/food/event.json?search=date_started:[20040101+TO+20240101]&limit=10'

    # Send a GET request to the API using the requests library
    api_response = requests.get(api_url)

    # Check if the API request was successful
    if api_response.status_code == 200:

        # Inform the user that the data was received
        print("Data received from the API.")

        # Parse the JSON response data and store it in "api_data"
        api_data = api_response.json()

        # Check if the "results" key exists in the API response
        if 'results' in api_data:

            # Normalize the JSON data
            df = pd.json_normalize(api_data['results'])

            # Display the number of records and columns in the DataFrame
            print(f"API data loaded with {df.shape[0]} records and {df.shape[1]} columns.")

        else:
            # Handle the case where the "results" key is missing in the API response
            print("Error: 'results' key not found in the API response.")

            # Set df to None to indicate that no valid data was loaded
            df = None
    else:
        # Handle the case where the API request was not successful
        print(f"Error receiving data from API. Status code: {api_response.status_code}")

        # Set df to None to indicate that no valid data was loaded
        df = None




# Check if data was loaded successfully

# If no data was loaded
if df is None:
    # Inform the user that no data was loaded and exit the program
    print("No data loaded. Now exiting the program.")
else:
    # Display the columns of the loaded data
    print(f"\nThe dataset currently has {df.shape[1]} columns: {list(df.columns)}")
    print(f"Number of records: {df.shape[0]}, Number of columns: {df.shape[1]}")




# Modify Column(s): Remove Column(s)
    # Display the number of columns and column names to help the user select which to remove
    print(f"\nThe dataset currently has {df.shape[1]} columns.")
    print(f"Here are the names of the columns: {list(df.columns)}")

    # Loop until the user provides valid columns to remove
    while True:
        # Get user input for column(s) to remove, split by commas
        columns_to_remove = input("Enter the column names to remove (comma-separated): ").split(",")

        # Remove extra spaces from the column names
        columns_to_remove = [col.strip() for col in columns_to_remove]

        # Check if all entered columns exist in the DataFrame
        if all(col in df.columns for col in columns_to_remove):

            # Remove the specified columns from the DataFrame
            df_removed = df.drop(columns=columns_to_remove)
            print(f"Removed {len(columns_to_remove)} column(s).")

            # Exit the loop once valid columns have been entered
            break
        else:
            # Inform the user that one or more columns entered do not exist
            print("Error: One or more columns entered do not exist. Please try again.")



# Handle Unsupported Data Types
    print("Checking for unsupported data types:")

    # Display the data types of all columns in the DataFrame
    print(df_removed.dtypes)

    # Convert unsupported data types to strings
    for col in df_removed.columns:
        # If a column contains lists or dictionaries, convert them to strings
        if df_removed[col].apply(lambda x: isinstance(x, (list, dict))).any():
            df_removed[col] = df_removed[col].apply(str)
            print(f"Column '{col}' contained unsupported types and was converted to string.")

    # Display summary after modifications
    print(f"\nSummary of the dataset after removing column(s):")
    print(f"Number of records: {df_removed.shape[0]}")
    print(f"Number of columns: {df_removed.shape[1]}")





# Ask User for Output Format
    while True:

        # Prompt the user to choose the output format
        print("Choose the output format: 1 for CSV, 2 for JSON, 3 for SQL database")

        # Get input from the user and store it in the "output_format" variable
        output_format = input("Enter your choice (1, 2, or 3): ")

        # Check if the input is either "1", "2", or "3"
        if output_format in ['1', '2', '3']:

            # If valid, break out of the loop
            break
        else:

            # If invalid, inform the user and loop again
            print("Error: Invalid choice. Please enter 1 for CSV, 2 for JSON, or 3 for SQL database.")



# Save the Data Based on User Choice

    # If the user chooses CSV
    if output_format == '1':
        # Name of the output CSV file
        output_csv_converted = 'converted_output_data.csv'

        # Save the DataFrame to a CSV file
        df_removed.to_csv(output_csv_converted, index=False)

        # Inform the user that the file was saved successfully
        print(f"Converted dataset saved as CSV file: {output_csv_converted}")

    # If the user chooses JSON
    elif output_format == '2':
        # Name of the output JSON file
        output_json_converted = 'converted_output_data.json'

        # Save the DataFrame to a JSON file
        df_removed.to_json(output_json_converted, orient='records', indent=4)

        # Inform the user that the file was saved successfully
        print(f"Converted dataset saved as JSON file: {output_json_converted}")


    # If the user chooses SQL database
    elif output_format == '3':
        # Name of the output SQL database file
        output_sql_converted = 'converted_output_data.db'

        # Connect to the SQL database
        conn = sqlite3.connect(output_sql_converted)

        # Save the DataFrame to the SQL database
        df_removed.to_sql('converted_data', conn, if_exists='replace', index=False)

        # Inform the user that the data was successfully saved to the database
        print(f"Converted dataset saved to SQL database: {output_sql_converted}")

        # Close the database connection
        conn.close()

Choose the data source: 1 for CSV, 2 for API
Enter your choice (1 or 2): 0
Error: Invalid choice. Please enter 1 for CSV or 2 for API.
Choose the data source: 1 for CSV, 2 for API
Enter your choice (1 or 2): 2
Data received from the API.
API data loaded with 10 records and 9 columns.

The dataset currently has 9 columns: ['report_number', 'outcomes', 'date_created', 'reactions', 'date_started', 'products', 'consumer.age', 'consumer.age_unit', 'consumer.gender']
Number of records: 10, Number of columns: 9

The dataset currently has 9 columns.
Here are the names of the columns: ['report_number', 'outcomes', 'date_created', 'reactions', 'date_started', 'products', 'consumer.age', 'consumer.age_unit', 'consumer.gender']
Enter the column names to remove (comma-separated): sos
Error: One or more columns entered do not exist. Please try again.
Enter the column names to remove (comma-separated): outcomes
Removed 1 column(s).
Checking for unsupported data types:
report_number        object
date